In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [3]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_11월.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [4]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [ ]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

In [9]:
import pickle
# 데이터프레임을 피클 파일로 저장함으로써 데이터타입 보존
with open('11월_문장분리.pkl', 'wb') as file:
    pickle.dump(df, file)

In [3]:
import pickle
# 저장된 피클 파일을 불러오기 익일 재실행 경우, 이 코드만 돌리면 시간절약 가능.
with open('11월_문장분리.pkl', 'rb') as file:
    df = pickle.load(file)
df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/1004foot/222942729197,20.38km 걸은 날 과천 서울대공원...,2022.11.30.,#과천서울대공원 #딜쿠샤 #철도노조파업\n2022. 11. 30. 수요일\n오늘 과...,0,과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친구 ...,[(과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친...
1,https://blog.naver.com/kissyilove/222942724949,강화도 아뜰리에 카페화실,2022.11.30.,서울 근교의 강화도에서\n예쁜 카페를 만났어요~\n멋진 아뜰리에가 있어서\n그림 좋...,1,서울 근교의 강화도에서 예쁜 카페를 만났어요 멋진 아뜰리에가 있어서 그림 좋아하시는...,[(서울 근교의 강화도에서 예쁜 카페를 만났어요 멋진 아뜰리에가 있어서 그림 좋아하...
2,https://blog.naver.com/youngppong2/222942725728,계당기념관 탐방기[상명대학교],2022.11.30.,안녕하세요 여러분!!\n벌써 2022년이 마무리 되어가는데요🥹\n얼마전에 상명대학교...,2,안녕하세요 여러분 벌써 2022년이 마무리 되어가는데요 얼마전에 상명대학교 계당기념...,[(안녕하세요 여러분 벌써 2022년이 마무리 되어가는데요 얼마전에 상명대학교 계당...
3,https://blog.naver.com/kamuy46/222942724526,원주 Ak문화아카데미 오감달팽이 종강,2022.11.30.,어느덧 오감달팽이 가을학기 마지막 수업을 마쳤어요.\n그간 아팠던 쓰롱이는 두 번의...,3,어느덧 오감달팽이 가을학기 마지막 수업을 마쳤어요 그간 아팠던 쓰롱이는 두 번의 수...,"[(어느덧 오감달팽이 가을학기 마지막 수업을 마쳤어요, 0, 27, None, []..."
4,https://blog.naver.com/gochanggyu/222942723965,국태민안 제례를 올리는 동악단... 소나무숲 아름다운 국형사!,2022.11.30.,시내에 있어 접근성이 뛰어난 치악산 국형사는 시민들이 자주 찾는 사찰입니다.\n전통...,4,시내에 있어 접근성이 뛰어난 치악산 국형사는 시민들이 자주 찾는 사찰입니다 전통적인...,"[(시내에 있어 접근성이 뛰어난 치악산 국형사는 시민들이 자주 찾는 사찰입니다, 0..."
...,...,...,...,...,...,...,...
22954,https://blog.naver.com/j_____10/222916175028,10月,2022.11.01.,안녕하셔요\n제가 또 돌아왔네요\n10월도 긴 것 같은데 기분 탓인가\n10월 첫날...,22954,안녕하셔요 제가 또 돌아왔네요 10월도 긴 것 같은데 기분 탓인가 10월 첫날부터 ...,"[(안녕하셔요, 0, 5, None, []), (제가 또 돌아왔네요, 6, 16, ..."
22955,https://blog.naver.com/tjsdu8320/222916174317,혼자 다녀온 경주 불국사 템플스테이 후기 (숙소부터 일정까지!),2022.11.01.,경주 불국사 템플스테이 후기\n참된 나를 찾아가는 행복여행\n원래 8월 말에 휴가 ...,22955,경주 불국사 템플스테이 후기 참된 나를 찾아가는 행복여행 원래 8월 말에 휴가 시기...,[(경주 불국사 템플스테이 후기 참된 나를 찾아가는 행복여행 원래 8월 말에 휴가 ...
22956,https://blog.naver.com/mardians/222916052167,"[100대명산] 전남 순천 조계산 등산코스, 선암사 송광사 단풍...",2022.11.01.,조계산 - 888m\n1. 산림청 - 100대 명산\n2. 한국의산하 - 100대 ...,22956,조계산 888m 1 산림청 100대 명산 2 한국의산하 100대 명산 89위 3 B...,[(조계산 888m 1 산림청 100대 명산 2 한국의산하 100대 명산 89위 3...
22957,https://blog.naver.com/gnbab/222915197448,【분석과 해설】 ‘강남 불패’ 신화,2022.11.01.,현대자동차그룹 글로벌비즈니스센터(GBC) 조감도1\n*출처 : 글로벌비즈니스센터 -...,22957,현대자동차그룹 글로벌비즈니스센터 GBC 조감도1 출처 글로벌비즈니스센터 나무위키 n...,[(현대자동차그룹 글로벌비즈니스센터 GBC 조감도1 출처 글로벌비즈니스센터 나무위키...


In [4]:
# 데이터프레임을 10개 단위로 나누기
chunk_size = 50
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99
df_100
df_101
df_102
df_103
df_104
df_105
df_106
df_107
df_108
df_109
df_110
df_111
df_112
df_113
df_114
df_115
df_116
df_117
df_118
df_119
df_120
df_121
df_122
df_123
df_124
df_125
df_126
df_127
df_128
df_129
df_130
df_131
df_132
df_133
df_134
df_135
df_136
df_137
df_138
df_139
df_140
df_141
df_142
df_143
df_144
df_145
df_146
df_147
df_148
df_149
df_150
df_151
df_152
df_153
df_154
df_155
df_156
df_157
df_158
df

In [12]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_11월_전처리.csv", index=False, encoding='UTF-8')

In [5]:
import pandas as pd
df2 = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_11월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [ ]:
df2 = df2[df2['org_idx'] < 9650]

In [ ]:
df2['org_idx'].tolist()[-1]//50+2

In [ ]:
for a in range(df2['org_idx'].tolist()[-1]//50+2,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_11월_전처리.csv", index=False, encoding='UTF-8')
df2

In [8]:
df2.dropna(axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.to_csv("../dataset/2022/본문_전처리/나무문화_본문_6월_전처리.csv", index=False, encoding='UTF-8')